In [ ]:
import time
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import json
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import string
from itertools import islice
from selenium.webdriver.common.keys import Keys
# from Feed import Feed
from icecream import ic
from typing import Iterator, Any, List

In [21]:
from Surfer import GetBrowser,RedditFeed,Feed

driver = GetBrowser()

In [22]:
def uuid():
    letters_digits = string.ascii_letters
    return ''.join(random.choice(letters_digits) for i in range(20))

In [23]:
def get_script(fname:str):
  script = ''
  with open(f'./scripts/{fname}.js','r') as f:
    script = f.read()
  script = script.replace('\n','')
  def replacer(*args):
    acc = script
    for i,x in enumerate(args):
      acc = acc.replace('#'+str(i),str(x))
    return 'try{' + f'console.log(`{fname}`,JSON.parse(`{json.dumps(args)}`));'+acc+'}catch(err){console.log(`{fname}`,err);}'
  return replacer
def run_script(name:str,verbose=False):
	def script(*args:Any):
		script_string = get_script(name)(*args)
		answer = driver.execute_script(script_string)
		if verbose:
			print('script_string: ',script_string,' Answer: ', answer)
		return answer
	return script

In [24]:
def random_scroll(
  mean = 0.5
	, stddev = 0.01
):
	return max(0, min(random.normalvariate(mean, stddev) , 1))

In [25]:
class RedditFeed(Feed):
	def __init__(self, driver,skip=True,verbose=False):
		super()
		self.name = 'Reddit Feed'
		# Store the WebDriver instance
		self.driver = driver
		self.verbose = verbose
		if not skip:
			self.driver.get('https://www.reddit.com/')

		run_script('init_reddit',verbose=self.verbose)()

	def __iter__(self) -> Iterator[Any]:
		while True:
			if not run_script('check curser',verbose=self.verbose)():
				continue
			yield run_script('parse post',verbose=self.verbose)()
			run_script('reddit scroll next',verbose=self.verbose)(random_scroll(),random_scroll(),random.randint(1,25)/100)
			while run_script('check still scrolling')():
				time.sleep(0.1)

	def click_post(self,newTab=True):
		id = uuid()
		run_script('mark curser',verbose=self.verbose)(id)
		# Find the DOM element to click on
		sel = f'.{id}'
		element = driver.find_element(By.CSS_SELECTOR,sel)
  
		# Get the dimensions of the element
		element_size = element.size
		element_location = element.location

		# Calculate the x and y coordinates of the random point in the upper center of the element
		x = element_location["x"] + element_size["width"] / 2
		y = element_location["y"] + random.randint(0, element_size["height"] // 2)

		# Move the mouse to the calculated coordinates and click
		actions = ActionChains(driver)
		X = x - element_location["x"]
		Y = y - element_location["y"]
		ic(X)
		ic(Y)
		if newTab:
			actions.move_to_element_with_offset(element, X, Y) \
				.key_down(Keys.CONTROL) \
				.click() \
				.key_up(Keys.CONTROL) \
				.perform()
		else:
			actions.move_to_element_with_offset(element, X, Y) \
				.click() \
				.perform()


	def close(self):
		# Close the WebDriver instance
		self.driver.quit()

In [26]:
rf = RedditFeed(driver,skip=False)
# rf = RedditFeed(driver,skip=True)

WebDriverException: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=112.0.5615.138)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0084DCE3+50899]
	(No symbol) [0x007DE111]
	(No symbol) [0x006E5588]
	(No symbol) [0x006D8305]
	(No symbol) [0x006D820C]
	(No symbol) [0x006D6F06]
	(No symbol) [0x006D7668]
	(No symbol) [0x006E6D22]
	(No symbol) [0x0073E631]
	(No symbol) [0x0072B8FC]
	(No symbol) [0x0073E01C]
	(No symbol) [0x0072B6F6]
	(No symbol) [0x00707708]
	(No symbol) [0x0070886D]
	GetHandleVerifier [0x00AB3EAE+2566302]
	GetHandleVerifier [0x00AE92B1+2784417]
	GetHandleVerifier [0x00AE327C+2759788]
	GetHandleVerifier [0x008E5740+672048]
	(No symbol) [0x007E8872]
	(No symbol) [0x007E41C8]
	(No symbol) [0x007E42AB]
	(No symbol) [0x007D71B7]
	BaseThreadInitThunk [0x77D0FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77EB7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77EB7A

In [ ]:
time.sleep(4)
for x in islice(rf,5):
  print(x)
  time.sleep(2*max(1,len(x) * 0.01))
  rf.click_post()

In [ ]:
# decultter html
# Array.from(x.querySelectorAll('*')).map((x,i)=>{
#     const cl = Array.from(x.classList)
#     const to_remove = cl.filter(x=> !/^[a-z-]+$/.exec(x))
#     for (let cls of to_remove) {
#         x.classList.remove(cls)
#     }
#     if (x.className === ''){
#         x.removeAttribute('class')
#     }
# })